In [ ]:
# https://python.langchain.com/docs/use_cases/summarization

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

# 문서 로드(Load Documents)

In [3]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [4]:
# from langchain_community.document_loaders import DirectoryLoader
# 
# loader = DirectoryLoader(".", glob="data/*.pdf", show_progress=True)
# docs = loader.load()

In [5]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/개인정보 보호법(법률)(제19234호)(20240315).pdf")

# 페이지 별 문서 로드
docs = loader.load_and_split()

In [6]:
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[2500:2600])

문서의 수: 41

[메타데이터]

{'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'page': 0}

========= [앞부분] 미리보기 =========


# 프롬프트
* 프롬프트 엔지니어링
    - 주어진 데이터(context)를 토대로 우리가 원하는 결과를 도출할 때 중요한 역할을 함

# 언어모델 생성(Create LLM)

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import load_summarize_chain, LLMChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)


C:\Users\HP-15s\workspace\law-summary\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 69689 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
from langchain import hub

map_prompt = hub.pull("rlm/map-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [ ]:
# Reduce
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
from langchain.chains import MapReduceDocumentsChain

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    # 정말 작은 청크 크기를 설정합니다.
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
split_docs = recursive_text_splitter.split_documents(docs)


In [ ]:
print(map_reduce_chain.run(split_docs))